In [ ]:
from os import listdir
from os.path import isfile, join

import string

import progressbar
from time import sleep

from string import punctuation

import pymorphy2
from nltk.corpus import stopwords

import re

from gensim.models.keyedvectors import FastTextKeyedVectors

import numpy as np
from collections import OrderedDict

from scipy.spatial import distance
import operator

import statistics

full_punctuation = punctuation + "–" + "," + "»" + "«"
full_punctuation

In [ ]:
def check_metastring(line):
    """отбрасываем строки без текста"""
    for char in line:
        if char != " " and char not in full_punctuation and char.isalpha() :
            return line
    return ''

In [ ]:
morph = pymorphy2.MorphAnalyzer()

def lemmatize(text,morph):
"""лемматизируем строки"""
    lemm_split_text = []
    tokens = text.split()
    #print(tokens)
    
    clean_text = []
    for word in tokens:
        if word not in russian_stopwords and word != " ":
            clean_word = ''
            for char in word:
                if char not in punctuation:
                    clean_word+= char
            if(clean_word):
                clean_text.append(clean_word)
    
   # print(clean_text)
    
    for word in clean_text:
        parsed_word = morph.parse(word)[0]
        lemma = parsed_word.normal_form
        
        lemm_split_text.append(lemma)
    return ' '.join(lemm_split_text)

In [ ]:
def preprocess_text(cleaned_text):
    """лемматизируем текст"""
    lemm_text = []
    bar = progressbar.ProgressBar(maxval=len(cleaned_text),
                                  widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    lines = 0

    
    for line in cleaned_text:
        t = lemmatize(line, morph)
        lemm_text.append(t)

        lines += 1
        bar.update(lines)
        sleep(0.1) 
        
    return lemm_text

In [ ]:
def get_clean_and_lemm_csv(file, filename):
    """на вход принимается файл с субтитрами и его название, на выходе получаем датасет с оригинальным и лемматизированным текстом, 
    разделенным в соответсвтие с моментом произношения"""
    print("NOW HANDLING ", file)
    
    cleaned_text = []
    with open(file, "r", encoding = "cp1251") as sub:
        
        phrase = []
        parse_full_phrase = False
        try:
            for orig_line in sub.readlines():
                line = check_metastring(orig_line)
                #print("<||", line,"||>")

                if not line and phrase:
                    output_string = ''
                    #print(phrase)
                    for p_text in phrase:
                        #print(p_text)
                        p_text = re.sub(' +', ' ', p_text)
                        p_text = p_text.rstrip()
                        p_text = p_text.lstrip()
                        output_string +=  p_text +' '
                    output_string = output_string.rstrip()

                    if output_string != '':
                        cleaned_text.append(output_string)
                    phrase = []
                else:
                    phrase.append(line)
        except:
            print("looks like encoding error")
            return None, None
                
    lemm_text = preprocess_text(cleaned_text)
    
    processed_file = pd.DataFrame(
    {'cleaned_text': cleaned_text,
     'lemm_text': lemm_text,
    })
    
    filename = filename + ".csv"
    print("going to save", filename)
    processed_file.to_csv(filename, sep='\t', encoding='cp1251', index=False)
    return cleaned_text, lemm_text 

In [ ]:
for item in listdir('./sub/subs_all/'):
    if item.endswith(".srt"):
        print (join('./sub/subs_all', item))
        get_clean_and_lemm_csv(join('./sub/subs_all', item), item)